Estructuras de control. Diccionarios. Archivos JSON. ¿A dónde voy en las vacaciones de invierno? Gratis…

Este proyecto consiste en permitir el registro y consulta de eventos gratuitos que se realicen dentro del territorio argentino. Está dirigido a dos tipos de usuarios.

Para aquellos que deseen dar a conocer un evento, indicando día, hora y lugar de realización, como así también el nombre y descripción del evento.
Para aquellos que deseen consultar la oferta de eventos en su municipio o un listado de eventos a realizarse en un periodo de tiempo.
El programa iniciará ofreciendo un menú (Registro/Consulta). Registro solicitará los datos necesarios para generar un archivo JSON a partir del diseño de diccionarios con la siguiente forma: { “nombre_evento”: “Festival por aniversario del Club Atlético Libertad”, "descripcion": “Evento familiar con músicos locales”, "dia": 12, "mes": “julio”, "hora": 15, "provincia": “Salta”, "localidad": “Tartagal”, }

Los datos “provincia” y “localidad” deben ser extraídos de las correspondientes APIs. Es decir, debe mostrarse al usuario una lista de provincias y a partir de su elección, una lista de las localidades de esa provincia.

Consulta ofrecerá otro menú (Eventos en mi localidad/Todos los eventos/Eventos entre dos fechas).

Para “Eventos en mi localidad”, nuevamente accederemos a la información de las APIs para conocer provincia y localidad del usuario y luego buscaremos dentro del archivo JSON ya generado todos los eventos vinculados.

Para “Todos los eventos” simplemente debemos listar completamente la información del archivo JSON generado.

Para “Eventos entre dos fechas” solicitamos al usuario día y mes de inicio y día y mes de fin del intervalo que desea conocer y se realizará una búsqueda dentro del JSON creado que cumpla con los requisitos pedidos.

Para realizar esta actividad contamos con las siguientes URLs

https://apis.datos.gob.ar/georef/api/provincias?campos=id,nombre

https://apis.datos.gob.ar/georef/api/municipios?provincia=22&c ampos=id,nombre&max=100

In [ ]:
import requests
import json

# Función para obtener la lista de provincias desde la API
def obtener_provincias():
    url = "https://apis.datos.gob.ar/georef/api/provincias?campos=id,nombre"
    response = requests.get(url)
    data = response.json()
    provincias = {provincia["id"]: provincia["nombre"] for provincia in data["provincias"]}
    return provincias

# Función para obtener la lista de localidades de una provincia desde la API
def obtener_localidades(provincia_id):
    url = f"https://apis.datos.gob.ar/georef/api/municipios?provincia={provincia_id}&campos=id,nombre&max=100"
    response = requests.get(url)
    data = response.json()
    localidades = {localidad["id"]: localidad["nombre"] for localidad in data["municipios"]}
    return localidades

# Función para registrar un evento en el archivo JSON
def registrar_evento():
    nombre_evento = input("Ingrese el nombre del evento: ")
    descripcion = input("Ingrese la descripción del evento: ")
    dia = int(input("Ingrese el día de realización del evento: "))
    mes = input("Ingrese el mes de realización del evento: ")
    hora = int(input("Ingrese la hora de realización del evento: "))

    provincias = obtener_provincias()
    for id_provincia, nombre_provincia in provincias.items():
        print(f"{id_provincia}: {nombre_provincia}")

    provincia_id = input("Seleccione el ID de la provincia donde se realizará el evento: ")
    localidades = obtener_localidades(provincia_id)
    for id_localidad, nombre_localidad in localidades.items():
        print(f"{id_localidad}: {nombre_localidad}")

    localidad_id = input("Seleccione el ID de la localidad donde se realizará el evento: ")

    evento = {
        "nombre_evento": nombre_evento,
        "descripcion": descripcion,
        "dia": dia,
        "mes": mes,
        "hora": hora,
        "provincia": provincias[provincia_id],
        "localidad": localidades[localidad_id]
    }

    with open('eventos.json', 'a') as archivo:
        json.dump(evento, archivo)
        archivo.write("\n")

    print("Evento registrado correctamente.")

# Función para consultar eventos desde el archivo JSON
def consultar_eventos():
    print("1. Eventos en mi localidad")
    print("2. Todos los eventos")
    print("3. Eventos entre dos fechas")

    opcion = int(input("Seleccione una opción: "))

    if opcion == 1:
        provincias = obtener_provincias()
        for id_provincia, nombre_provincia in provincias.items():
            print(f"{id_provincia}: {nombre_provincia}")

        provincia_id = input("ID de la Provincia: ")

        localidades = obtener_localidades(provincia_id)
        for id_localidad, nombre_localidad in localidades.items():
            print(f"{id_localidad}: {nombre_localidad}")

        localidad_id = input("ID de la Localidad: ")

        # Obtener las localidades y provincia desde la API


        eventos = []
        with open('eventos.json', 'r') as archivo:
            for linea in archivo:
                evento = json.loads(linea)
                if evento["provincias"] == provincia.get(provincia_id, "") and evento["localidad"] == localidades.get("localidad_id", ""):
                    eventos.append(evento)

        if eventos:
            print("Eventos en tu localidad: ")
            for evento in eventos:
                print(evento)
        else:
            print("No se encontraron eventos en tu localidad.")

    elif opcion == 2:
        with open('eventos.json', 'r') as archivo:
            eventos = [json.loads(linea) for linea in archivo]
            if eventos:
                print("Todos los eventos: ")
                for evento in eventos:
                    print("No hay eventos registrados.")

    elif opcion == 3:
        # Obtener fechas de inicio y fin del usuario
        dia_inicio = int(input("Día de inicio (número): "))
        mes_inicio = input("Mes de Inicio: ")
        dia_fin = int(input("Día de fin(número): "))

        eventos = []
        with open('eventos.json', 'r') as archivo:
            for linea in archivo:
                evento = json.loads(linea)
                if evento['mes'] == mes_inicio or (evento['mes'] > mes_inicio and evento['mes'] < mes_fin) or (evento['mes'] == mes_fin and evento['dia'] <= dia_fin):
                    eventos.append(evento)
        if eventos:
            print("Eventos entre las fechas especificadas: ")
            for evento in eventos:
                print(evento)
        else:
          print("No se encontraron eventos en el intervalo especificado.")
    else:
        print("Opción inválida.")

# Función principal que muestra el menú principal
def main():
    while True:
        print("Menú Principal")
        print("1. Registro de evento")
        print("2. Consulta de eventos")
        print("3. Salir")
        opcion = int(input("Seleccione una opción: "))

        if opcion == 1:
            registrar_evento()
        elif opcion == 2:
            consultar_eventos()
        elif opcion == 3:
            break
        else:
            print("Opción inválida.")

if __name__ == "__main__":
    main()


Menú Principal
1. Registro de evento
2. Consulta de eventos
3. Salir
Seleccione una opción: 1
Ingrese el nombre del evento: Fogon de los trabajadores
Ingrese la descripción del evento: Folclore en vivo
Ingrese el día de realización del evento: 03
Ingrese el mes de realización del evento: 08
Ingrese la hora de realización del evento: 22
54: Misiones
74: San Luis
70: San Juan
30: Entre Ríos
78: Santa Cruz
62: Río Negro
26: Chubut
14: Córdoba
50: Mendoza
46: La Rioja
10: Catamarca
42: La Pampa
86: Santiago del Estero
18: Corrientes
82: Santa Fe
90: Tucumán
58: Neuquén
66: Salta
22: Chaco
34: Formosa
38: Jujuy
02: Ciudad Autónoma de Buenos Aires
06: Buenos Aires
94: Tierra del Fuego, Antártida e Islas del Atlántico Sur
Seleccione el ID de la provincia donde se realizará el evento: 66
660084: La Merced
660035: Las Lajitas
660252: Tolar Grande
660056: Cafayate
660077: Cerrillos
660133: Embarcación
660280: Río Piedras
660119: General Güemes
660343: Santa Victoria Este
660364: Campo Quijano
66

KeyError: ignored

In [ ]:

import requests
import json

def obtener_provincias():
    url = "https://apis.datos.gob.ar/georef/api/provincias?campos=id,nombre"
    response = requests.get(url)
    data = response.json()
    provincias = {provincia["id"]: provincia["nombre"] for provincia in data["provincias"]}
    return provincias

def obtener_localidades(provincia):
    url = f"https://apis.datos.gob.ar/georef/api/municipios?provincia={provincia}&campos=id,nombre&max=100"
    response = requests.get(url)
    data = response.json()
    localidades = {localidad["id"]: localidad["nombre"] for localidad in data["municipios"]}
    return localidades

def registrar():
    nombre_evento= input("Ingrese el nombre del evento: ")
    descripcion= input("Ingrese la descripción del evento: ")
    dia=int(input("Ingrese el día de realización del evento: "))
    mes= input("Ingrese el mes de realización del evento: ")
    hora= int(input("Ingrese la hora de realización del evento: "))

    provincias = obtener_provincias()
    for id_provincia, nombre_provincia in provincias.items():
        print(f"{id_provincia}: {nombre_provincia}")

    provincia_id = int(input("Seleccione el ID de la provincia donde se realizará el evento: "))
    localidades = obtener_localidades(provincia_id)
    for id_localidad, nombre_localidad in localidades.items():
        print(f"{id_localidad} {nombre_localidad}")

    localidad_id = int(input("Seleccione el ID de la localidad donde se realizará el evento: "))

    datos_evento = {
        "nombre_evento": nombre_evento,
        "descripcion": descripcion,
        "dia": dia,
        "mes": mes,
        "hora": hora,
        "provincia": provincias[provincia_id],
        "localidad": localidades[localidad_id]
    }

    with open("eventos.json", "a") as archivo:
      json.dump(datos_evento, archivo)
      archivo.write("\n")

    print("Evento registrado correctamente.")

def consultar_eventos():
    print("1. Eventos en mi localidad")
    print("2. Todos los eventos")
    print("3. Eventos entre dos fechas")
    opcion = int(input("Seleccione una opción: "))

    if opcion == 1:
        # Implementar consulta de eventos en la localidad del usuario
        pass
    elif opcion == 2:
        # Implementar listado de todos los eventos
        pass
    elif opcion == 3:
        # Implementar búsqueda de eventos entre dos fechas
        pass
    else:
        print("Opción inválida.")

def menu():
  while True:
    print("--------------------------------------------------")
    print("       Eventos gratuitos                          ")
    print("--------------------------------------------------")
    print("1.Consultar eventos gratuitos disponibles")
    print("2.Registrar eventos gratuitos")
    print("3.Salir")
    opcion=int(input("Ingrese la opción deseada: "))


    if opcion==1:
      consultar_eventos()
    elif opcion==2:
      registrar()
    else:
        if opcion==3:
          break
        else:
          print ("la opción ingresada no es correcta")


if __name__ == "__main__":
    menu()


--------------------------------------------------
       Eventos gratuitos                          
--------------------------------------------------
1.Consultar eventos gratuitos disponibles
2.Registrar eventos gratuitos
3.Salir
1. Eventos en mi localidad
2. Todos los eventos
3. Eventos entre dos fechas
--------------------------------------------------
       Eventos gratuitos                          
--------------------------------------------------
1.Consultar eventos gratuitos disponibles
2.Registrar eventos gratuitos
3.Salir
